In [2]:
!pip install sendgrid 

  Obtaining dependency information for sendgrid from https://files.pythonhosted.org/packages/c2/9c/45d068fd831a65e6ed1e2ab3233de58784842afdc62fdcdd0a01bbb6b39d/sendgrid-6.12.4-py3-none-any.whl.metadata
  Obtaining dependency information for python-http-client>=3.2.1 from https://files.pythonhosted.org/packages/29/31/9b360138f4e4035ee9dac4fe1132b6437bd05751aaf1db2a2d83dc45db5f/python_http_client-3.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for ecdsa<1,>=0.19.1 from https://files.pythonhosted.org/packages/cb/a3/460c57f094a4a165c84a1341c373b0a4f5ec6ac244b998d5021aade89b77/ecdsa-0.19.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for werkzeug>=1.0.0 from https://files.pythonhosted.org/packages/52/24/ab44c871b0f07f491e5d2ad12c9bd7358e527510618cb1b803a88e986db1/werkzeug-3.1.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/102.1 kB ? eta -:--:--
   ---------------- ----------------------- 41.0/102.1 kB ? eta -:--:--
   -----


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import requests
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

def check_endpoint(endpoint, params, api_key):
    headers = {"Authorization": f"Bearer {api_key}"}
    apology_message = "Apologies, I am unable to answer this question right now. Something unexpected happened, try asking again!"
    success = False
    error_message = ""

    for _ in range(3):
        try:
            res = requests.get(endpoint, json=params, headers=headers)

            if res.text.strip().endswith(apology_message):
                error_message = "Server Error"
                continue

            if res.status_code != 200:
                error_message = f"JSON STATUS CODE: {res.status_code}"
                continue

            success = True
            error_message = ""
            break

        except Exception as e:
            error_message = f"Exception: {str(e)}"
            continue

    return "pass" if success else "fail", error_message

def send_email(subject, body):
    message = Mail(
        from_email=os.environ.get("ALERT_FROM_EMAIL"),
        to_emails=os.environ.get("ALERT_TO_EMAIL"),
        subject=subject,
        plain_text_content=body
    )
    try:
        sg = SendGridAPIClient(os.environ.get("SENDGRID_API_KEY"))
        sg.send(message)
    except Exception as e:
        print(f"Email send failed: {e}")

if __name__ == "__main__":
    endpoints = {
        "Dev v7": "https://dev.bg-app-insights.com/v7",
        "UAT v7": "https://uat.bg-app-insights.com/v7",
        "Prod v7": "https://prod.bg-app-insights.com/v7",
        "Dev v8": "https://dev.bg-app-insights.com/v8",
        "UAT v8": "https://uat.bg-app-insights.com/v8",
        "Prod v8": "https://prod.bg-app-insights.com/v8"
    }

    api_keys = {
        "Dev v7": "abcdefgh",
        "UAT v7": "t31VJnwkQ4GEC75D0j2YBxeEO",
        "Prod v7": "IX6DUsnStDyre41lbTzQwViv8",
        "Dev v8": "abcdefgh",
        "UAT v8": "t31VJnwkQ4GEC75D0j2YBxeEO",
        "Prod v8": "IX6DUsnStDyre41lbTzQwViv8"
    }

    params = {
        "list_messages": [{"role": "user", "content": "What is zanu?"}],
        "date_filter": [{"start": "2020-01-01", "end": "2025-12-31"}],
        "allowed_data_sources": [],
        "persona": "coba",
        "therapeutic_area": "",
        "username": "beghou",
        "country": []
    }
    # api_key = "abcdefgh"

    for env_name, endpoint in endpoints.items():
        api_key = api_keys[env_name]
        status, error = check_endpoint(endpoint, params, api_key)

        if status == "fail":
            print(f"[{env_name}] Check failed: {error}")
            # send_email(
            #     subject=f"🚨 API Monitor Alert - {env_name} (FAILED)",
            #     body=f"The {env_name} API check failed for endpoint {endpoint}.\n\nError: {error}"
            # )
        else:
            print(f"[{env_name}] Check passed.")

[Dev v7] Check passed.
[UAT v7] Check passed.
[Prod v7] Check passed.
[Dev v8] Check passed.
[UAT v8] Check passed.
[Prod v8] Check passed.
